# Silver Layer Transformations

In [1]:
# coding=utf-8
from pyspark.sql import SparkSession
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
spark = SparkSession.builder.config("spark.driver.memory", "15g").getOrCreate()

### Load Data

In [3]:
# There are csv files with data from military draft since 2010
years = []
for file in os.listdir("../Raw"):
    f = file.split(".")[0] # Removing the extension .csv
    f = int(f.replace("sermil", "")) # Getting only the year

    years.append(f)

print(years)

[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]


In [4]:
# The csv files have the same columns, so I'm creating a big dataframe and adding them to it

sermil_df = None
for y in years:
    print(f"Iniciando ano {y}")
    df = spark.read.option("header",True).csv(f"../Raw/sermil{y}.csv") # Opening the file from the year y

    # Since the year of 2022 there is no information about religion anymore, so Im removing it
    if "RELIGIAO" in df.columns:
        df = df.drop("RELIGIAO")

    if sermil_df == None:
        sermil_df = df
    else:
        sermil_df = sermil_df.union(df)

Iniciando ano 2010
Iniciando ano 2011
Iniciando ano 2012
Iniciando ano 2013
Iniciando ano 2014
Iniciando ano 2015
Iniciando ano 2016
Iniciando ano 2017
Iniciando ano 2018
Iniciando ano 2019
Iniciando ano 2020
Iniciando ano 2021
Iniciando ano 2022
Iniciando ano 2023


In [5]:
sermil_df.limit(10).toPandas()

,ANO_NASCIMENTO,PESO,ALTURA,CABECA,CALCADO,CINTURA,MUN_NASCIMENTO,UF_NASCIMENTO,PAIS_NASCIMENTO,ESTADO_CIVIL,...,ESCOLARIDADE,VINCULACAO_ANO,DISPENSA,ZONA_RESIDENCIAL,MUN_RESIDENCIA,UF_RESIDENCIA,PAIS_RESIDENCIA,JSM,MUN_JSM,UF_JSM
0,1984,None,160.0,None,None,71.0,PORTO VELHO,RO,BRASIL,Solteiro,...,9ª Série do Ensino Fundamental,2010,Sem dispensa,Urbana,RIO BRANCO,AC,BRASIL,31/72 - RIO BRANCO,RIO BRANCO,AC
1,1973,None,None,None,None,None,PRESIDENTE DUTRA,MA,BRASIL,Solteiro,...,Ensino Médio Completo,2010,Com dispensa,Urbana,QUATIPURU,PA,BRASIL,27/139 - ACAILANDIA,ACAILANDIA,MA
2,1973,None,None,None,None,None,LAGO DA PEDRA,MA,BRASIL,Solteiro,...,7ª Série do Ensino Fundamental,2010,Com dispensa,Rural,QUATIPURU,PA,BRASIL,27/139 - ACAILANDIA,ACAILANDIA,MA
3,1989,68.0,175.0,58.0,43.0,80.0,TENENTE ANANIAS,RN,BRASIL,Solteiro,...,2ª Série do Ensino Médio,2010,Sem dispensa,Urbana,NATAL,RN,BRASIL,24/81 - NATAL CENTRO,NATAL,RN
4,1991,None,None,None,None,None,NATAL,RN,BRASIL,Solteiro,...,1ª Série do Ensino Médio,2010,Sem dispensa,Urbana,NATAL,RN,BRASIL,24/81 - NATAL CENTRO,NATAL,RN
5,1973,None,None,None,None,None,IMPERATRIZ,MA,BRASIL,Solteiro,...,1ª Série do Ensino Médio,2010,Com dispensa,Rural,QUATIPURU,PA,BRASIL,27/139 - ACAILANDIA,ACAILANDIA,MA
6,1973,None,None,None,None,None,SANTO ANTONIO DOS LOPES,MA,BRASIL,Solteiro,...,3ª Série do Ensino Fundamental,2010,Com dispensa,Rural,QUATIPURU,PA,BRASIL,27/139 - ACAILANDIA,ACAILANDIA,MA
7,1973,None,None,None,None,None,PEDREIRAS,MA,BRASIL,Solteiro,...,3ª Série do Ensino Fundamental,2010,Com dispensa,Urbana,QUATIPURU,PA,BRASIL,27/139 - ACAILANDIA,ACAILANDIA,MA
8,1973,None,None,None,None,None,BELO HORIZONTE,MG,BRASIL,Solteiro,...,5ª Série do Ensino Fundamental,2010,Com dispensa,Urbana,BELO HORIZONTE,MG,BRASIL,11/22 - BELO HORIZONTE,BELO HORIZONTE,MG
9,1973,None,None,None,None,None,PALHANO,CE,BRASIL,Solteiro,...,6ª Série do Ensino Fundamental,2010,Com dispensa,Urbana,QUATIPURU,PA,BRASIL,27/139 - ACAILANDIA,ACAILANDIA,MA


### Transforming the Data

In [6]:
from pyspark.sql.functions import regexp_replace, when, col
from pyspark.sql import functions as F

# Removing accents
sermil_df = sermil_df.withColumn("ESCOLARIDADE", F.translate("ESCOLARIDADE", 'ãáéíóôúÁÉÍÓÔÚ',
                                                                             'aaeioouAEIOOU'))

In [7]:
# Adjusting the schooling information
sermil_df = sermil_df.withColumn(
                "ESCOLARIDADE",
                when(
                    col("ESCOLARIDADE").rlike(r"(Semestre|Ano|Serie).*"),  # Take the lines with incomplete schooling
                    regexp_replace(col("ESCOLARIDADE"), r".* (-|do) (.*)", r"$2 Incompleto")  # Substitute by the incomplete schooling
                ).otherwise(col("ESCOLARIDADE"))  # Otherwise keep as is
)

In [8]:
# Correting manually this bugged one
sermil_df = sermil_df.withColumn("ESCOLARIDADE",
                                 when(col("ESCOLARIDADE") == "Pos-Graduaç╞o", "Pos-Graduaçao").otherwise(col("ESCOLARIDADE")))

In [9]:
sermil_df.createOrReplaceTempView("sermil_df")

spark.sql("""
SELECT ESCOLARIDADE, count(*) as QTD
FROM sermil_df
GROUP BY ESCOLARIDADE
ORDER BY ESCOLARIDADE
""").toPandas()

,ESCOLARIDADE,QTD
0,Alfabetizado,203857
1,Analfabeto,116585
2,Doutorado,425
3,Ensino Fundamental Completo,750814
4,Ensino Fundamental Incompleto,4727672
5,Ensino Medio (Profissionalizante) Incompleto,68731
6,Ensino Medio Completo,4569576
7,Ensino Medio Incompleto,9180035
8,Ensino Superior Completo,59028
9,Ensino Superior Incompleto,1118618


### Saving the data

In [10]:
# Saving as parquet, partitioning by the year of military draft
sermil_df.write.parquet("../Silver/sermil.parquet", mode="overwrite", partitionBy="VINCULACAO_ANO")